# Importing Data & Packages

In [1]:
#pip install pandas

In [2]:
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras 
from keras import layers

import random

print('Everything Imported Succesfully!')

Everything Imported Succesfully!


In [3]:
data=pd.read_csv('/kaggle/input/301-330-days/Data_5IAL_3_301.csv',index_col=0,parse_dates=[1])
data.rename(columns={'0':'Date'},inplace=True)
data['5IAL_3_301.BatchName']=data['5IAL_3_301.BatchName'].fillna('No Batch Specified')
data.head()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Date,5IAL_3_TT301.50,5IAL_3_QIT301.52,5IAL_3_PIT 301.55,5IAL_3_QIT301.57,5IAL_3_PIT301.60,5IAL_3_FIT301.61MF,5IAL_3_FIT301.61VF,5IAL_3_FIT301.61D,5IAL_3_PIT301.63,...,5IAL_3_GSC301.44,5IAL_3_GSO301.44,5IAL_3_LSL301.64,5IAL_3_LSL301.68,5IAL_3_LSLL301.69,5IAL_3_301.OCCUPIED,5IAL_3_LIT301.54,5IAL_3_LSH301.56,5IAL_3_XPV301.13,5IAL_3_WY301.54
0,2022-04-10 12:00:00,21.80,9.84,-0.02,0.0,0.0,0.0,0.0,0.00105,2.07,...,0,0,0,0,0,0,1.40,0,0.0,0.0
1,2022-04-10 12:01:00,21.80,9.84,-0.02,0.0,0.0,0.0,0.0,0.00106,2.07,...,0,0,0,0,0,0,1.40,0,0.0,0.0
2,2022-04-10 12:02:00,21.80,9.84,-0.02,0.0,0.0,0.0,0.0,0.00102,2.07,...,0,0,0,0,0,0,1.41,0,0.0,0.0
3,2022-04-10 12:03:00,21.78,9.84,-0.02,0.0,0.0,0.0,0.0,0.00105,2.07,...,0,0,0,0,0,0,1.40,0,0.0,0.0
4,2022-04-10 12:04:00,21.78,9.84,-0.02,0.0,0.0,0.0,0.0,0.00104,2.07,...,0,0,0,0,0,0,1.40,0,0.0,0.0


# Data Splitting and Grouping 

In [4]:
Batch_Names=data['5IAL_3_301.BatchName']
unique_names=Batch_Names.unique()
print(unique_names)

['No Batch Specified' 'YP5812210101' 'YP5812210102' ... 'KB212306501'
 'NA KB212306501' 'KC112306501']


In [5]:
print(unique_names.size)

1629


In [6]:
#Convert the unique_names numpy array to a pandas series
temp_df=pd.Series(unique_names)

# Create a boolean mask to select rows where the length of the specified column is between 11 and 12 characters (inclusive)
mask = (temp_df.str.len() >= 11) & (temp_df.str.len() <= 12)

# Use the boolean mask to select only the rows that meet the criteria
Viable_Batches = unique_names[mask]

print(Viable_Batches.size)

del temp_df

1205


## Train Test Split

Randomly pick the Test_Train_split fraction of data from the filterd unique name list.\
This way a percentage of batches are not used in training and thus can be used for testing.

In [7]:
To_split=Viable_Batches
Test_Train_split=0.2

Test_Batches=random.sample(To_split.tolist(), round(To_split.size*Test_Train_split))

Train_Batches = [x for x in To_split if x not in Test_Batches]

print(len(Test_Batches))
print(len(Train_Batches))

241
964


In [8]:
Train_Data=data[data['5IAL_3_301.BatchName'].isin(Train_Batches)]
Test_Data=data[data['5IAL_3_301.BatchName'].isin(Test_Batches)]

In [9]:
Train_Data.head()

,Date,5IAL_3_TT301.50,5IAL_3_QIT301.52,5IAL_3_PIT 301.55,5IAL_3_QIT301.57,5IAL_3_PIT301.60,5IAL_3_FIT301.61MF,5IAL_3_FIT301.61VF,5IAL_3_FIT301.61D,5IAL_3_PIT301.63,...,5IAL_3_GSC301.44,5IAL_3_GSO301.44,5IAL_3_LSL301.64,5IAL_3_LSL301.68,5IAL_3_LSLL301.69,5IAL_3_301.OCCUPIED,5IAL_3_LIT301.54,5IAL_3_LSH301.56,5IAL_3_XPV301.13,5IAL_3_WY301.54
1280,2022-04-11 09:20:00,26.81,6.64,0.04,10.16,-0.00,0.00,0.00,0.00161,2.07,...,0,1,0,0,0,1,-0.47,0,0.00,0.00
1281,2022-04-11 09:21:00,26.74,6.65,0.04,10.01,0.00,0.00,0.00,0.00163,2.07,...,0,1,0,0,0,1,3.72,0,0.00,20.40
1282,2022-04-11 09:22:00,26.50,9.44,1.06,124.95,0.01,119.98,6.97,1.03350,2.07,...,1,0,0,0,0,1,18.02,0,1.97,191.99
1283,2022-04-11 09:23:00,26.97,9.73,1.10,172.66,0.01,116.62,6.81,1.02800,2.07,...,1,0,0,0,0,1,29.51,0,2.00,375.84
1284,2022-04-11 09:24:00,27.36,9.90,0.63,189.06,0.00,151.06,8.77,1.03390,2.07,...,1,0,0,0,0,1,32.60,0,2.02,502.56


## Normalise the selection with use of the Z-Score
The Z-score can be described as:\
$Z=\frac{x-\mu}{\sigma}$

In this x is your value, μ is the average of the population that value is taken from, and σ is the standard deviation of the population. It is basicaly a measure of how far a given value is away from its mean in standard deviations. To give you a good feeling, in a normal distribution 66% of values are within 1 std of the mean. 99% are withing 2 std and 99.9 is within 3

Note that i am not yet sure how representative all of this is for the data, it just allows me to use Massflow and resulting pressure more easaly because the range of values are simalar and i dont have to specify two seperate axis with values within 1 figure.

In [10]:
data_norm=pd.DataFrame()
data_norm_values=pd.DataFrame()

names=Train_Data['5IAL_3_301.BatchName']

col_2_drop=['Date','5IAL_3_301.BatchName','5IAL_3_LSH301.56','5IAL_3_XPV301.05','5IAL_3_XPV301.06','5IAL_3_XPV301.08','5IAL_3_XPV301.09','5IAL_3_XPV301.22','5IAL_3_XPV301.36','5IAL_3_XPV301.42','5IAL_3_XPV301.43','5IAL_3_XPV301.46']

df=Train_Data.drop(col_2_drop,axis=1)

data_norm=(df-df.mean())/df.std()

for col in df:
    # zval=(Train_Grouped[col]-np.mean(Train_Grouped[col]))/np.std(Train_Grouped[col])
    # data_norm=pd.concat([data_norm,zval],axis=1)
    data_norm_values[col]=[np.mean(df[col]),np.std(df[col])]

data_norm=pd.concat([data_norm,names],axis=1)


In [11]:
data_norm.head()
#data_norm_values.head()

,5IAL_3_TT301.50,5IAL_3_QIT301.52,5IAL_3_PIT 301.55,5IAL_3_QIT301.57,5IAL_3_PIT301.60,5IAL_3_FIT301.61MF,5IAL_3_FIT301.61VF,5IAL_3_FIT301.61D,5IAL_3_PIT301.63,5IAL_3_P301.70,...,5IAL_3_GSC301.44,5IAL_3_GSO301.44,5IAL_3_LSL301.64,5IAL_3_LSL301.68,5IAL_3_LSLL301.69,5IAL_3_301.OCCUPIED,5IAL_3_LIT301.54,5IAL_3_XPV301.13,5IAL_3_WY301.54,5IAL_3_301.BatchName
1280,-0.241426,-1.559876,-1.202862,-0.301085,-0.093938,-1.500014,-0.022841,-3.166018,-0.834709,-1.532669,...,-3.219144,3.221224,-0.0161,-0.11615,-0.043135,0.061556,-2.033289,-1.532361,-2.049460,YP5812210102
1281,-0.265541,-1.552595,-1.202862,-0.301098,-0.093938,-1.500014,-0.022841,-3.165953,-0.834709,-1.532669,...,-3.219144,3.221224,-0.0161,-0.11615,-0.043135,0.061556,-1.904142,-1.532361,-2.018089,YP5812210102
1282,-0.348223,0.478667,1.634073,-0.291164,7.395618,0.334215,-0.009654,0.157744,-0.834709,0.658504,...,0.310639,-0.310438,-0.0161,-0.11615,-0.043135,0.061556,-1.463381,0.625521,-1.754222,YP5812210102
1283,-0.186305,0.689801,1.745325,-0.287041,7.395618,0.282848,-0.009956,0.140029,-0.834709,0.669460,...,0.310639,-0.310438,-0.0161,-0.11615,-0.043135,0.061556,-1.109231,0.658382,-1.471501,YP5812210102
1284,-0.051946,0.813570,0.438110,-0.285623,-0.093938,0.809360,-0.006248,0.159033,-0.834709,0.603724,...,0.310639,-0.310438,-0.0161,-0.11615,-0.043135,0.061556,-1.013989,0.680290,-1.276634,YP5812210102


In [12]:
data_norm.isna().sum()

5IAL_3_TT301.50         0
5IAL_3_QIT301.52        0
5IAL_3_PIT 301.55       0
5IAL_3_QIT301.57        0
5IAL_3_PIT301.60        0
5IAL_3_FIT301.61MF      0
5IAL_3_FIT301.61VF      0
5IAL_3_FIT301.61D       0
5IAL_3_PIT301.63        0
5IAL_3_P301.70          0
5IAL_3_R301.71          0
5IAL_3_P301.72          0
5IAL_3_XPV301.35        0
5IAL_3_XPV301.53        0
5IAL_3_XPV301.54        0
5IAL_3_XPV301.63        0
5IAL_3_LSL301.51        0
5IAL_3_LSL301.53        0
5IAL_3_GSC301.44        0
5IAL_3_GSO301.44        0
5IAL_3_LSL301.64        0
5IAL_3_LSL301.68        0
5IAL_3_LSLL301.69       0
5IAL_3_301.OCCUPIED     0
5IAL_3_LIT301.54        0
5IAL_3_XPV301.13        0
5IAL_3_WY301.54         0
5IAL_3_301.BatchName    0
dtype: int64

In [13]:


Train_Grouped=data_norm.groupby(["5IAL_3_301.BatchName"],as_index=True)
                                 
Train_Grouped.size()

5IAL_3_301.BatchName
CB212211501     179
CB212211502     124
CB212215401     111
CB212215402     113
CB212220801     179
               ... 
YP7012304802    142
kC112219301     136
kP5512213801    170
kp5502226203    154
kp7012226605    136
Length: 964, dtype: int64

In [14]:
# AU_Train=np.empty([111,39,0])


# for row in Train_Batches:
#     batch_row=Train_Grouped.get_group(row)
#     expand_dim=np.expand_dims(batch_row,axis=2)
#     np.append(AU_Train,expand_dim,axis=2)


# print(AU_Train.shape())

In [15]:
columns=27
batches=964

model = keras.Sequential(
    [
        layers.Input(shape=(columns,1,1)),
        layers.Conv2D(
            filters=32, kernel_size=8, padding="same", strides=1, activation="relu"
        ),
        layers.Dropout(rate=0.1),
        layers.Conv2D(
            filters=16, kernel_size=16, padding="same", strides=1, activation="relu"
        ),
        layers.Conv2DTranspose(
            filters=16, kernel_size=16, padding="same", strides=1, activation="relu"
        ),
        layers.Dropout(rate=0.1),
        layers.Conv2DTranspose(
            filters=32, kernel_size=8, padding="same", strides=1, activation="relu"
        ),
        layers.Conv2DTranspose(filters=1, kernel_size=6, padding="same"),
    ]
)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="mse")
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 27, 1, 32)         2080      
                                                                 
 dropout (Dropout)           (None, 27, 1, 32)         0         
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 1, 16)         131088    
                                                                 
 conv2d_transpose (Conv2DTra  (None, 27, 1, 16)        65552     
 nspose)                                                         
                                                                 
 dropout_1 (Dropout)         (None, 27, 1, 16)         0         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 27, 1, 32)        32800     
 ranspose)                                              

In [16]:
for row in Train_Batches:
    X_batch=Train_Grouped.get_group(row)
    X_batch=X_batch.drop(['5IAL_3_301.BatchName'],axis=1)
    X_batch=np.expand_dims(X_batch,axis=2)
    X_batch=np.expand_dims(X_batch,axis=3)
    print(X_batch.shape)
    history = model.fit(
        X_batch,
        X_batch,
        epochs=50,
        verbose=0,
        validation_split=0.1,
        callbacks=[
            keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min")
        ],
    )
    print(row)

(103, 27, 1, 1)


2023-04-04 12:47:10.555302: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


YP5812210102
(221, 27, 1, 1)
CD112210101
(104, 27, 1, 1)
CD112210102
(101, 27, 1, 1)
CD112210103
(205, 27, 1, 1)
CD112210201
(137, 27, 1, 1)
KD112210201
(195, 27, 1, 1)
KD112210202
(127, 27, 1, 1)
KD112210203
(132, 27, 1, 1)
KD112210204
(99, 27, 1, 1)
KD112210205
(155, 27, 1, 1)
KD112210206
(214, 27, 1, 1)
KD112210207
(104, 27, 1, 1)
MD112210301
(184, 27, 1, 1)
MD112210302
(229, 27, 1, 1)
MD112210303
(123, 27, 1, 1)
MD112210305
(95, 27, 1, 1)
MD112210306
(122, 27, 1, 1)
YD112210301
(106, 27, 1, 1)
YD112210402
(122, 27, 1, 1)
YD112210403
(117, 27, 1, 1)
YD112210404
(99, 27, 1, 1)
YD112210405
(149, 27, 1, 1)
YD112210406
(242, 27, 1, 1)
YD112210407
(125, 27, 1, 1)
YP7012210401
(123, 27, 1, 1)
YP7012210502
(97, 27, 1, 1)
YP7012210901
(83, 27, 1, 1)
YP7012210902
(96, 27, 1, 1)
KP7012210901
(91, 27, 1, 1)
KP7012210903
(85, 27, 1, 1)
KP7012210904
(90, 27, 1, 1)
KP7012210905
(120, 27, 1, 1)
KP7012210906
(112, 27, 1, 1)
KP7012211001
(133, 27, 1, 1)
KP7012211002
(88, 27, 1, 1)
KP7012211003
(198,

In [17]:
model.save("trained_2D_AU")

In [18]:
#x_train_pred = model.predict(x_train)